# OpenCV DNN - Part 1

- OpenCV DNN
- Yolo Darknet
- OpenCV DNN Yolo
- Train Yolo Model with Custom Dataset

- Original Source : https://github.com/alitourani/yolo-license-plate-detection

In [1]:
import cv2
import numpy as np
import os

In [2]:
cv2.__version__

'4.2.0'

## Intro Deep Learning

- “#1 Pengenalan Machine Learning”  https://medium.com/@yunusmuhammad007/pengenalan-machine-learning-2320b5ca7266
- “#2 Supervised VS Unsupervised VS Reinforcement ML”  https://link.medium.com/wEcGRj3gq5
- “#3 Machine Learning Evaluation”  https://link.medium.com/qJ9Kd26gq5
- “#4 Alat dan Bahan untuk Machine Learning”  https://medium.com/@yunusmuhammad007/3-alat-dan-bahan-untuk-machine-learning-92c717286624
- “#5 Basic Python Programming”  https://medium.com/@yunusmuhammad007/5-basic-python-programming-87c89e1d0d3e
- “#6 Artificial Neural Network (ANN) — Part 1 (Pengenalan)”  https://link.medium.com/TbaRUcJZv5
- “#7 Artificial Neural Network (ANN) — Part 2 (Single Layer Perceptron)”  https://link.medium.com/kpBiXHBdz5
- “#8 Artificial Neural Network (ANN) — Part 3 (Teori Dasar Multi Layer Perceptron Backpropagation)”  https://link.medium.com/D7rAjn69F6
- “#9 Artificial Neural Network (ANN) — Part 4 (MLP Backpropagation Time Series Forecasting…”  https://link.medium.com/s2ZZFy89F6
- “#10 Artificial Neural Network (ANN) — Part 5 (Time Series Forecasting ISPU CO DKI Jakarta…”  https://link.medium.com/ccHKkBaaG6
- “#11 Artificial Neural Network (ANN) — Part 6 Konsep Dasar Convolutional Neural Network (CNN)”  https://link.medium.com/gy2J4beaG6

## OpenCV DNN

- Compatibility : > OpenCV 3.3
- Wiki : https://github.com/opencv/opencv/wiki/Deep-Learning-in-OpenCV
- Since OpenCV 3.1 there is DNN module in the library that implements **forward pass** (inferencing) with deep networks, **pre-trained**using some popular deep learning frameworks.
- The supported frameworks:
    - Caffe
    - TensorFlow
    - Torch
    - Darknet (Yolo)
    - Models in ONNX format

## Yolo (neural networks for object detection)

https://github.com/AlexeyAB/darknet#pre-trained-models

- Paper Yolo v4: https://arxiv.org/abs/2004.10934
- More details: [medium link](https://medium.com/@alexeyab84/yolov4-the-most-accurate-real-time-neural-network-on-ms-coco-dataset-73adfd3602fe?source=friends_link&sk=6039748846bbcf1d960c3061542591d7)
- Manual: https://github.com/AlexeyAB/darknet/wiki
- Result : https://www.youtube.com/user/pjreddie/videos <br>
<img src="resource/opencv_dnn.gif" style="width:500px"></img>
<br>
<img src="resource/82835867-f1c62380-9ecd-11ea-9134-1598ed2abc4b.png" style="width:700px"></img>

### OpenCV Yolo DNN

- Required files :
    - `.names` file
    - `.cfg` file
    - `.weights` file  
- **cfg-files** - are structures of neural networks: https://github.com/AlexeyAB/darknet/tree/master/cfg
- **weights-files** - are weights for correspond cfg-file, can be downloaded from: https://pjreddie.com/darknet/

- Read class names from `.names` file

In [3]:
classesFile = "yolo/plate_number/plate_number.names"

classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')
    
print(classes)

['License Plate']


- Load `.cfg` and `.weights` using `cv2.dnn.readNetFromDarknet()`
- Download `.weights` file from [this](https://drive.google.com/file/d/1vXjIoRWY0aIpYfhj3TnPUGdmJoHnWaOc/)

In [4]:
modelConfiguration = "yolo/plate_number/plate_number_yolov3.cfg"
modelWeights = "yolo/plate_number/plate_number_yolov3.weights"

net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)

In [5]:
type(net)

cv2.dnn_Net

#### OpenCV DNN Backend
- Set OpenCV DNN Backend using `.setPreferableBackend(backend_type)`
- where `backend_type` :
    - `cv2.dnn.DNN_BACKEND_DEFAULT`
    - `cv2.dnn.DNN_BACKEND_VKOM`
    - `cv2.dnn.DNN_BACKEND_HALIDE`
    - `cv2.dnn.DNN_BACKEND_OPENCV`
    - `cv2.dnn.DNN_BACKEND_INFERENCE_ENGINE`
      
#### OpenCV DNN Target
- Set OpenCV DNN Target using `.setPreferableTarget(target_type)`
- where `target_type` :
    - `cv2.dnn.DNN_TARGET_CPU`
    - `cv2.dnn.DNN_TARGET_MYRIAD`
    - `cv2.dnn.DNN_TARGET_OPENCL`
    - `cv2.dnn.DNN_TARGET_FPGA`
    - `cv2.dnn.DNN_TARGET_VULKAN`
    - `cv2.dnn.DNN_TARGET_CUDA` > OpenCV 4.2

In [6]:
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

- Convert image to blob using `cv2.dnn.blobFromImage()`

In [8]:
inpWidth = 416      # Width of network's input image
inpHeight = 416     # Height of network's input image

In [43]:
frame = cv2.imread("dataset/mobil3.jpg")
blob = cv2.dnn.blobFromImage(
                        frame, 
                        1/255, 
                        (inpWidth, inpHeight), 
                        [0, 0, 0],
                        1, 
                        crop=False)

In [44]:
type(blob), blob.shape, blob.dtype

(numpy.ndarray, (1, 3, 416, 416), dtype('float32'))

In [45]:
frame.shape, frame.dtype

((194, 259, 3), dtype('uint8'))

- Get Output layer using `.getUnconnectedOutLayersNames()` from `net` object

In [46]:
net.getLayerNames()

['conv_0',
 'bn_0',
 'relu_1',
 'conv_1',
 'bn_1',
 'relu_2',
 'conv_2',
 'bn_2',
 'relu_3',
 'conv_3',
 'bn_3',
 'relu_4',
 'shortcut_4',
 'conv_5',
 'bn_5',
 'relu_6',
 'conv_6',
 'bn_6',
 'relu_7',
 'conv_7',
 'bn_7',
 'relu_8',
 'shortcut_8',
 'conv_9',
 'bn_9',
 'relu_10',
 'conv_10',
 'bn_10',
 'relu_11',
 'shortcut_11',
 'conv_12',
 'bn_12',
 'relu_13',
 'conv_13',
 'bn_13',
 'relu_14',
 'conv_14',
 'bn_14',
 'relu_15',
 'shortcut_15',
 'conv_16',
 'bn_16',
 'relu_17',
 'conv_17',
 'bn_17',
 'relu_18',
 'shortcut_18',
 'conv_19',
 'bn_19',
 'relu_20',
 'conv_20',
 'bn_20',
 'relu_21',
 'shortcut_21',
 'conv_22',
 'bn_22',
 'relu_23',
 'conv_23',
 'bn_23',
 'relu_24',
 'shortcut_24',
 'conv_25',
 'bn_25',
 'relu_26',
 'conv_26',
 'bn_26',
 'relu_27',
 'shortcut_27',
 'conv_28',
 'bn_28',
 'relu_29',
 'conv_29',
 'bn_29',
 'relu_30',
 'shortcut_30',
 'conv_31',
 'bn_31',
 'relu_32',
 'conv_32',
 'bn_32',
 'relu_33',
 'shortcut_33',
 'conv_34',
 'bn_34',
 'relu_35',
 'conv_35',
 'b

In [47]:
layerOutput = net.getUnconnectedOutLayersNames()

In [48]:
layerOutput

['yolo_82', 'yolo_94', 'yolo_106']

- Set blob to input network using `.setInput()` on `net` object
- Do forward pass and get output using `.forward()` on `net` object

In [49]:
net.setInput(blob)
outs = net.forward(layerOutput)

In [50]:
outs[0][0][5:]

array([0.], dtype=float32)

In [51]:
classId = np.argmax(outs[0][0][5:])

In [52]:
# for out in outs:
frame_h, frame_w, frame_c = frame.shape
classIds = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        classId = np.argmax(scores)
        confidence = scores[classId]
        c_x = int(detection[0] * frame_w)
        c_y = int(detection[1] * frame_h)
        w = int(detection[2] * frame_w)
        h = int(detection[3] * frame_h)
        x = int(c_x - w / 2)
        y = int(c_y - h / 2)
        classIds.append(classId)
        confidences.append(float(confidence))
        boxes.append([x, y, w, h])

- Find Non maximum supression (NMS) from boxes using `cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)`

In [53]:
confThreshold = 0.5  # confidence level threshold
nmsThreshold = 0.4 # NMS threshold

In [54]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
for i in indices:
    i = i[0]
    box = boxes[i]
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]

    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
cv2.imshow("frame", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Wrap all into function

In [57]:
def drawPred(classId, conf, left, top, right, bottom):
    cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 255), 2)

    label = '%.2f' % conf
    if classes:
        assert(classId < len(classes))
        label = '%s: %s' % (classes[classId], label)

    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv2.rectangle(frame, (left, top - round(1.5*labelSize[1])), (max(right, left + labelSize[0]), top + baseLine), (255, 0, 255), -1)
    cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.50, (255, 255, 255), 1)

def postprocess(frame, outs):
    frame_h, frame_w, frame_c = frame.shape
    classIds = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:        
                scores = detection[5:]
                classId = np.argmax(scores)
                confidence = scores[classId]
                c_x = int(detection[0] * frame_w)
                c_y = int(detection[1] * frame_h)
                w = int(detection[2] * frame_w)
                h = int(detection[3] * frame_h)
                x = int(c_x - w / 2)
                y = int(c_y - h / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        drawPred(classIds[i], confidences[i], left,
                 top, left + width, top + height)


In [58]:
def draw_inf_time(frame, label, left =25, top=50):
    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    cv2.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (0, 255, 255), -1)
    cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    return frame

In [60]:
dataset = "dataset/" 
layerOutput = net.getUnconnectedOutLayersNames()
for filename in os.listdir(dataset):
    if os.path.splitext(filename)[1] != ".jpg" :
        continue
    frame = cv2.imread(os.path.join(dataset, filename))
    blob = cv2.dnn.blobFromImage(
        frame, 1/255, (inpWidth, inpHeight), [0, 0, 0], 1, crop=False)

    net.setInput(blob)
    outs = net.forward(layerOutput)

    postprocess(frame, outs)

    t, _ = net.getPerfProfile()
    label = 'Inference time: %.2f s' % (t / cv2.getTickFrequency())

    frame = draw_inf_time(frame, label)
    
    cv2.imshow("out", frame)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture("cars_on_the_road_2.mp4")
while cap.isOpened :
    ret, frame = cap.read()
    
    if not ret:
        continue
    blob = cv2.dnn.blobFromImage(
        frame, 1/255, (inpWidth, inpHeight), [0, 0, 0], 1, crop=False)

    net.setInput(blob)
    outs = net.forward(getOutputsNames(net))

    postprocess(frame, outs)

    t, _ = net.getPerfProfile()
    label = 'Inference time: %.2f s' % (t / cv2.getTickFrequency())
    cv2.putText(frame, label, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))

    cv2.imshow("out", frame)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

# Train Yolo Using Your Custom Dataset

- **Prepare Dataset** : 100 image car (just for testing, in real case, need a lot of data)
- **Dataset Annotation** : To annotante dataset using Yolo Format, use this desktop app : **[LabelImg]( https://github.com/tzutalin/labelImg)**
<img src="resource/demo3.jpg" style="width:600px"></img>
- **Train Model** on Colab ( https://colab.research.google.com/drive/1lTGZsfMaGUpBG4inDIQwIJVW476ibXk_#scrollTo=WewV3jU3B4Eo )